In [1]:
import pandas as pd
import folium
import numpy as np
import matplotlib.cm as cm

In [2]:
daten=pd.read_csv('Points.csv', sep=';')
daten.head(10)

,Point,Polygon Nr,Longitude,Latitude,Name,Pr-Wert
0,Point 1,1,"6,106032729","50,76884994",Neumarkt,1
1,Point 2,1,"6,106241941","50,7681205",Neumarkt,1
2,Point 3,1,"6,106815934","50,76819514",Neumarkt,1
3,Point 4,1,"6,1065799","50,76892458",Neumarkt,1
4,Point 5,1,"6,106032729","50,76884994",Neumarkt,1
5,Point 6,2,"6,10234201","50,76832746",Frankenberger Park,2
6,Point 7,2,"6,102873087","50,76763194",Frankenberger Park,2
7,Point 8,2,"6,103135943","50,7676014",Frankenberger Park,2
8,Point 9,2,"6,10311985","50,76745212",Frankenberger Park,2
9,Point 10,2,"6,104117632","50,76743855",Frankenberger Park,2


In [3]:
def komma2punkt(value):
    try:
        value = value.replace(',','.')
        return value
    except ValueError:
        return value

In [4]:
k2p_cols = ['Longitude','Latitude']
daten[k2p_cols] = daten[k2p_cols].applymap(lambda x: komma2punkt(x))
#daten['Longitude'] = daten['Longitude'].apply(komma2punkt)
#daten['Latitude'] = daten['Latitude'].apply(lambda x: komma2punkt(x))
daten

,Point,Polygon Nr,Longitude,Latitude,Name,Pr-Wert
0,Point 1,1,6.106032729,50.76884994,Neumarkt,1
1,Point 2,1,6.106241941,50.7681205,Neumarkt,1
2,Point 3,1,6.106815934,50.76819514,Neumarkt,1
3,Point 4,1,6.1065799,50.76892458,Neumarkt,1
4,Point 5,1,6.106032729,50.76884994,Neumarkt,1
5,Point 6,2,6.10234201,50.76832746,Frankenberger Park,2
6,Point 7,2,6.102873087,50.76763194,Frankenberger Park,2
7,Point 8,2,6.103135943,50.7676014,Frankenberger Park,2
8,Point 9,2,6.10311985,50.76745212,Frankenberger Park,2
9,Point 10,2,6.104117632,50.76743855,Frankenberger Park,2


In [5]:
def polygon_df_to_geojson(df, properties, lat='Latitude', lon='Longitude'):
    """
    Turn a dataframe containing polygon (by "Polygon Nr") data into a geojson formatted python dictionary

    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """

    # find polygon groups
    group = df.groupby(['Polygon Nr']).groups.keys()

    # create a new python dict to contain our geojson data, using geojson format
    geojson = {"type": "FeatureCollection", "features": []}

    for i in range(len(group)):
        new_df = df.loc[df['Polygon Nr'] == [x for x in df.groupby(['Polygon Nr']).groups.keys()][i]]
        res_df = new_df.reset_index(drop=True)
        coords = []
        # create a feature template to fill in
        feature = {"type": "Feature",
                   "properties": {},
                   "geometry": {"type": "Polygon",
                                "coordinates": []}}
        # loop through each row in the dataframe and convert each row to geojson format
        for j in range(len(res_df)):
            coords.append([float(res_df[lon][j]), float(res_df[lat][j])])

        # fill in the coordinates
        feature["geometry"]["coordinates"] = [coords]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = str(res_df[prop][i])
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson["features"].append(feature)

    return geojson

In [6]:
keys = list(daten['Pr-Wert'].unique())
#color_range = list(np.linspace(0, 1, len(keys), endpoint=False))
#colors = [cm.tab20b(x) for x in color_range]
colors = ['red','green','yellow']
color_dict = dict(zip(keys, colors))

In [7]:
color_dict[1]

'red'

In [8]:
out=polygon_df_to_geojson(daten, ['Name','Pr-Wert'])
out['features'][0]['properties']['Pr-Wert']

'1'

In [9]:
cols=[color_dict[daten['Pr-Wert'][i]] for i in range(len(daten))]
cols

['red',
 'red',
 'red',
 'red',
 'red',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'green',
 'yellow',
 'yellow',
 'yellow',
 'yellow',
 'yellow']

In [10]:
datensatz=polygon_df_to_geojson(daten, ['Name','Pr-Wert'])
m = folium.Map(
    location=[50.77,6.10],
    zoom_start=15,
    #tiles='Mapbox Bright'
)
folium.GeoJson(datensatz,
    style_function=lambda x :{
        'color' : 'red' if x['properties']['Pr-Wert']=='1' else 'orange' if x['properties']['Pr-Wert']=='2' else 'green',
        'opacity': 0.6,
        'fillColor':'red' if x['properties']['Pr-Wert']=='1' else 'orange' if x['properties']['Pr-Wert']=='2' else 'green'}
        ).add_to(m)
m

In [11]:
polygon_df_to_geojson(daten, ['Name','Pr-Wert'])

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'Name': 'Neumarkt', 'Pr-Wert': '1'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[6.106032729, 50.76884994],
      [6.106241941, 50.7681205],
      [6.106815934, 50.76819514],
      [6.1065799, 50.76892458],
      [6.106032729, 50.76884994]]]}},
  {'type': 'Feature',
   'properties': {'Name': 'Frankenberger Park', 'Pr-Wert': '2'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[6.10234201, 50.76832746],
      [6.102873087, 50.76763194],
      [6.103135943, 50.7676014],
      [6.10311985, 50.76745212],
      [6.104117632, 50.76743855],
      [6.104262471, 50.76774729],
      [6.104112267, 50.7677914],
      [6.104101539, 50.76857513],
      [6.10234201, 50.76832746]]]}},
  {'type': 'Feature',
   'properties': {'Name': 'Boendgen', 'Pr-Wert': '3'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[6.099472046, 50.7669398],
      [6.098914146, 50.76650551],
      [6.101810932, 50.7

In [12]:
def write_to_file(data,filename):
    f= open(filename,"w+")
    f.write(str(data).replace("'",'"'))
    f.close
    print('{} successfully written to {}'.format(data,filename))

In [13]:
write_to_file(polygon_df_to_geojson(daten, ['Name','Pr-Wert']),'Frankenberger.geojson')

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'Name': 'Neumarkt', 'Pr-Wert': '1'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[6.106032729, 50.76884994], [6.106241941, 50.7681205], [6.106815934, 50.76819514], [6.1065799, 50.76892458], [6.106032729, 50.76884994]]]}}, {'type': 'Feature', 'properties': {'Name': 'Frankenberger Park', 'Pr-Wert': '2'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[6.10234201, 50.76832746], [6.102873087, 50.76763194], [6.103135943, 50.7676014], [6.10311985, 50.76745212], [6.104117632, 50.76743855], [6.104262471, 50.76774729], [6.104112267, 50.7677914], [6.104101539, 50.76857513], [6.10234201, 50.76832746]]]}}, {'type': 'Feature', 'properties': {'Name': 'Boendgen', 'Pr-Wert': '3'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[6.099472046, 50.7669398], [6.098914146, 50.76650551], [6.101810932, 50.76582014], [6.102122068, 50.76626801], [6.099472046, 50.7669398]]]}}]} successfully written to Frankenberger.geojson


In [14]:
dats=pd.read_json('Frankenberger.geojson'); dats

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'Name': 'Ne..."
1,FeatureCollection,"{'type': 'Feature', 'properties': {'Name': 'Fr..."
2,FeatureCollection,"{'type': 'Feature', 'properties': {'Name': 'Bo..."


In [15]:
def point_df_to_geojson(df, properties, lat='Latitude', lon='Longitude'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary
    
    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """
    
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson